*Assignment 3 297.201*



Group 11
- Amaz Salman 24001042
- Daniel Soung-Yun Kim 24016358
- Forest Bennett 23010240
- Karolina Cardenas Rios 20014550

In [7]:
from unittest.mock import inplace

from IPython.display import display, HTML
from docutils.nodes import abbreviation
from werkzeug.routing.rules import Weighting
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
import requests                 # How Python gets the webpages
from bs4 import BeautifulSoup   # Creates structured, searchable object
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import classification_report, brier_score_loss
import matplotlib.pyplot as plt


In [3]:
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score


In [4]:
from pylab import rcParams

rcParams['figure.dpi'] = 150
rcParams['lines.linewidth'] = 1.2
rcParams['axes.facecolor'] = 'white'
rcParams['patch.edgecolor'] = 'white'
rcParams['font.family'] = 'DejaVu Sans'
rcParams['figure.figsize'] = 5,3
rcParams['font.size'] = 10

In [5]:
rcParams['axes.labelsize'] = 'medium'
rcParams['xtick.labelsize'] = 8
rcParams['ytick.labelsize'] = 8

In [ ]:
#Amaz
def get_fighter_links():
    base_url = "http://www.ufcstats.com/statistics/fighters?char={}&page=all"
    letters = 'abcdefghijklmnopqrstuvwxyz'
    fighter_links = []

    for letter in letters:
        url = base_url.format(letter)
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table', class_='b-statistics__table')
            if table:
                links = table.find_all('a')
                for link in links:
                    href = link.get('href')
                    if href and 'fighter-details' in href:
                        fighter_links.append(href)
            time.sleep(0.5)
        except Exception as e:
            print(f"Ran into an error while checking letter {letter}: {e}")

    return list(set(fighter_links))


In [ ]:
#Amaz
def get_fighter_data(fighter_url):
    response = requests.get(fighter_url, timeout=10)
    soup = BeautifulSoup(response.content, 'html.parser')

    name = soup.find('span', class_='b-content__title-highlight')
    name = name.text.strip() if name else None

    record = soup.find('span', class_='b-content__title-record')
    record = record.text.strip().replace('Record: ', '') if record else None

    stats = soup.find_all('li', class_='b-list__box-list-item b-list__box-list-item_type_block')
    data = {'Name': name, 'Record': record, 'Fighter URL': fighter_url}

    for stat in stats:
        parts = stat.text.strip().split(':')
        if len(parts) == 2:
            key = parts[0].strip()
            value = parts[1].strip()
            data[key] = value

    return data

In [ ]:
#Amaz
def main():
    print("Grabbing all the fighter profile links...")
    fighter_links = get_fighter_links()
    print(f"Found {len(fighter_links)} fighters.")

    fighters_data = []
    errors = []

    for link in tqdm(fighter_links):
        try:
            fighter = get_fighter_data(link)
            fighters_data.append(fighter)
        except Exception as e:
            print(f"Couldn’t get data from {link}: {e}")
            errors.append((link, str(e)))
        time.sleep(0.5)

    df = pd.DataFrame(fighters_data)
    df.to_csv("./csv/ufc_fighters_dataset.csv", index=False)
    print("Done! Saved everything to ufc_fighters_dataset.csv")

    if errors:
        print(f"Had trouble with {len(errors)} links. You might want to check the log.")

main()

In [ ]:
#Karolina
base_api_url = 'https://api.sportradar.com/mma/trial/v2/en'
headers = {
    "accept": "application/json",
    "x-api-key": 'FYcFfXYU39pCxyf5QAYnJXWNlQs0IBdBPNU6wloe'
}

In [ ]:
#Karolina
def get_all_seasons():
    url_seasons = base_api_url + "/seasons.json"
    response_seasons = requests.get(url_seasons, headers=headers)
    seasons_content = json.loads(response_seasons.content)
    seasons_values = seasons_content.get('seasons')
    seasons_result = []
    for season in seasons_values:
        start_date = season.get("start_date")
        season_name = season.get('name')
        seasons_result.append({
            'Date': start_date,
            'Name': season_name
        })
    return seasons_result


In [ ]:
#Karolina
def get_all_fights_by_date(date):
    url_fights = base_api_url + '/schedules/' + date + '/summaries.json'
    response_fights = requests.get(url_fights, headers=headers)
    fights_content = json.loads(response_fights.content)
    fights_values = fights_content.get('summaries')
    fights_results = []
    if fights_values is not None:
        for fight in fights_values:
            sport_event = fight.get('sport_event')
            sport_event_context = sport_event.get('sport_event_context')
            competition = sport_event_context.get('competition')
            competition_name = competition.get('name')
            competitors = sport_event.get('competitors')
            sport_event_status = fight.get('sport_event_status')
            event_status = sport_event_status.get('status')
            if event_status == 'closed':
                winner_id = sport_event_status.get('winner_id')
                final_round = sport_event_status.get('final_round')
                final_round_length = sport_event_status.get('final_round_length')
                method = sport_event_status.get('method')
                scheduled_length = sport_event_status.get('scheduled_length')
                weight_class = sport_event_status.get('weight_class')
                winner = None
                loser = None
                for compet in competitors:
                    if compet.get('id') == winner_id:
                        winner = compet
                    else:
                        loser = compet
                if winner is not None and loser is not None:
                    winner_name = winner.get('name')
                    winner_abbrev = winner.get('abbreviation')
                    winner_gender = winner.get('gender')
                    winner_qualifier = winner.get('qualifier')
                    loser_name = loser.get('name')
                    loser_abbrev = loser.get('abbreviation')
                    loser_gender = loser.get('gender')
                    loser_qualifier = loser.get('qualifier')
                    loser_id = loser.get('id')
                    statistics = fight.get('statistics')
                    if statistics is not None:
                        totals = statistics.get('totals')
                        if totals is not None:
                            stats_comp = totals.get('competitors')
                            winner_stats = None
                            loser_stats = None
                            for comp in stats_comp:
                                if comp.get('id') == winner_id:
                                    winner_stats = comp.get('statistics')
                                else:
                                    loser_stats = comp.get('statistics')
                            if winner_stats is not None and loser_stats is not None:
                                winner_control = winner_stats.get('control')
                                winner_knockdowns = winner_stats.get('knockdowns')
                                winner_significant_strike_percentage = winner_stats.get('significant_strike_percentage')
                                winner_significant_strikes = winner_stats.get('significant_strikes')
                                winner_significant_strikes_attempted = winner_stats.get('significant_strikes_attempted')
                                winner_submission_attempts = winner_stats.get('submission_attempts')
                                winner_takedown_percentage = winner_stats.get('takedown_percentage')
                                winner_takedowns = winner_stats.get('takedowns')
                                winner_takedowns_attempted = winner_stats.get('takedowns_attempted')
                                winner_total_strike_percentage = winner_stats.get('total_strike_percentage')
                                winner_total_strikes = winner_stats.get('total_strikes')
                                winner_total_strikes_attempted = winner_stats.get('total_strikes_attempted')
                                loser_control = loser_stats.get('control')
                                loser_knockdowns = loser_stats.get('knockdowns')
                                loser_significant_strike_percentage = loser_stats.get('significant_strike_percentage')
                                loser_significant_strikes = loser_stats.get('significant_strikes')
                                loser_significant_strikes_attempted = loser_stats.get('significant_strikes_attempted')
                                loser_submission_attempts = loser_stats.get('submission_attempts')
                                loser_takedown_percentage = loser_stats.get('takedown_percentage')
                                loser_takedowns = loser_stats.get('takedowns')
                                loser_takedowns_attempted = loser_stats.get('takedowns_attempted')
                                loser_total_strike_percentage = loser_stats.get('total_strike_percentage')
                                loser_total_strikes = loser_stats.get('total_strikes')
                                loser_total_strikes_attempted = loser_stats.get('total_strikes_attempted')

                                fights_results.append({
                                    "date": date,
                                    'competition' : competition_name,
                                    "winner": winner_name,
                                    'winner_abbreviation': winner_abbrev,
                                    'winner_gender': winner_gender,
                                    "winner_qualifier": winner_qualifier,
                                    'winner_id': winner_id,
                                    'winner_control': winner_control,
                                    'winner_knockdowns': winner_knockdowns,
                                    'winner_significant_strike_percentage': winner_significant_strike_percentage,
                                    'winner_significant_strikes': winner_significant_strikes,
                                    'winner_significant_strikes_attempted': winner_significant_strikes_attempted,
                                    'winner_submission_attempts': winner_submission_attempts,
                                    'winner_takedown_percentage': winner_takedown_percentage,
                                    'winner_takedowns': winner_takedowns,
                                    'winner_takedowns_attempted': winner_takedowns_attempted,
                                    'winner_total_strike_percentage': winner_total_strike_percentage,
                                    'winner_total_strikes': winner_total_strikes,
                                    'winner_total_strikes_attempted': winner_total_strikes_attempted,
                                    "loser": loser_name,
                                    'loser_abbreviation': loser_abbrev,
                                    'loser_gender': loser_gender,
                                    "loser_qualifier": loser_qualifier,
                                    'loser_id': loser_id,
                                    'loser_control': loser_control,
                                    'loser_knockdowns': loser_knockdowns,
                                    'loser_significant_strike_percentage': loser_significant_strike_percentage,
                                    'loser_significant_strikes': loser_significant_strikes,
                                    'loser_significant_strikes_attempted': loser_significant_strikes_attempted,
                                    'loser_submission_attempts': loser_submission_attempts,
                                    'loser_takedown_percentage': loser_takedown_percentage,
                                    'loser_takedowns': loser_takedowns,
                                    'loser_takedowns_attempted': loser_takedowns_attempted,
                                    'loser_total_strike_percentage': loser_total_strike_percentage,
                                    'loser_total_strikes': loser_total_strikes,
                                    'loser_total_strikes_attempted': loser_total_strikes_attempted,
                                    'final_round': final_round,
                                    'final_round_length': final_round_length,
                                    'method': method,
                                    'scheduled_length': scheduled_length,
                                    "weight_class": weight_class,
                                })

    return fights_results

In [ ]:
#Karolina
def get_all_fights_data():
    seasons = get_all_seasons()
    all_fights = []
    for season in seasons:
        season_fights = get_all_fights_by_date(season['Date'])
        all_fights.extend(season_fights)
    return all_fights


In [ ]:
#Karolina
all_fights_data = get_all_fights_data()
df_all_fights = pd.DataFrame(all_fights_data)
df_all_fights.to_csv('./csv/df_all_fights.csv', index= False)
df_all_fights

In [ ]:
#Karolina
all_fighters = pd.read_csv('./csv/ufc_fighters_dataset.csv')
all_fights = pd.read_csv('./csv/df_all_fights.csv')

In [ ]:
#Karolina
winners = all_fights[['winner', 'winner_id', 'weight_class']]
losers = all_fights[['loser', 'loser_id', 'weight_class']]
all_fighters['N_Name'] = ""
all_fighters['id'] = np.NaN
all_fighters['weight_class']=""
for i, f in all_fighters.iterrows():
    f_name = f['Name']
    n_parts = f_name.split()
    surname = n_parts.pop()
    first_name = ''.join(n_parts)
    full_name = surname + ', ' + first_name
    all_fighters.at[i,'N_Name'] = full_name
    w = winners[winners['winner']== full_name]
    if not w.empty:
        w_first = w.iloc[0]
        all_fighters.at[i,'id'] = w_first['winner_id']
        all_fighters.at[i, 'weight_class'] = w_first['weight_class']
    else:
        l = losers[losers['loser']== full_name]
        if not l.empty:
            l_first = l.iloc[0]
            all_fighters.at[i,'id'] = l_first['loser_id']
            all_fighters.at[i, 'weight_class'] = l_first['weight_class']


all_fighters = all_fighters.dropna(subset=['id'])
all_fighters.to_csv('./csv/all_fighters.csv', index=False)

In [ ]:
#Forest
data = pd.read_csv('./csv/df_all_fights.csv')
data

In [ ]:
#Forest
data['strike_accuracy_diff '] = data['winner_significant_strike_percentage'] - data['loser_significant_strike_percentage']
data['total_strike_accuracy_diff  '] = data['winner_total_strike_percentage'] - data['loser_total_strike_percentage']
data['knockdowns_diff'] = data['winner_knockdowns'] - data['loser_knockdowns']
data['takedown_accuracy_diff'] = data['winner_takedown_percentage'] - data['loser_takedown_percentage']
data['takedowns_diff'] = data['winner_takedowns'] - data['loser_takedowns']
data['submission_attempts_diff'] = data['winner_submission_attempts'] - data['loser_submission_attempts']
data.to_csv('./csv/df_all_fights.csv', index=False)

In [ ]:
#Daniel&Amaz
df = pd.read_csv('./csv/df_all_fights.csv')
df.columns = df.columns.str.strip()

num_cols = [
    'strike_accuracy_diff',
    'total_strike_accuracy_diff',
    'knockdowns_diff',
    'takedown_accuracy_diff',
    'takedowns_diff',
    'submission_attempts_diff'
]
cat_cols = ['weight_class', 'method']

df_clean = df[num_cols + cat_cols].dropna()

X_pos = df_clean.copy()
y_pos = np.ones(len(X_pos))

X_neg = df_clean.copy()
X_neg[num_cols] = -X_neg[num_cols]
y_neg = np.zeros(len(X_neg))

X = pd.concat([X_pos, X_neg], ignore_index=True)
y = np.concatenate([y_pos, y_neg])


In [ ]:
#Daniel&Amaz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocess = ColumnTransformer([
    ('scale', StandardScaler(), num_cols),
    ('encode', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

In [ ]:
#Daniel&Amaz
models_to_test = {
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

In [ ]:
#Daniel&Amaz
model_scores = {}

for label, model in models_to_test.items():
    pipeline = ImbPipeline([
        ('prep', preprocess),
        ('smote', SMOTE(random_state=42)),
        ('clf', model)
    ])

    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_test)

    f1 = f1_score(y_test, preds)
    model_scores[label] = f1

    print(f"== {label} ==")
    print(classification_report(y_test, preds, digits=3))
    print("-" * 50)

In [ ]:
#Daniel&Amaz
plt.figure(figsize=(8, 5))
bars = plt.bar(model_scores.keys(), model_scores.values())
plt.ylim(0, 1)
plt.title("F1 Scores for Each Model")
plt.ylabel("F1 Score")

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.02, f"{yval:.2f}", ha='center')

plt.tight_layout()
plt.show()

In [ ]:
#Daniel&Amaz
df = pd.read_csv('/.csv/df_all_fights.csv')
df.columns = df.columns.str.strip()

num_cols = [
    'strike_accuracy_diff',
    'total_strike_accuracy_diff',
    'knockdowns_diff',
    'takedown_accuracy_diff',
    'takedowns_diff',
    'submission_attempts_diff'
]

cat_cols = ['weight_class', 'method']

base_feats = df[num_cols + cat_cols].dropna()

y_win = np.ones(len(base_feats), dtype=int)

flipped_feats = base_feats.copy()
flipped_feats[num_cols] = -flipped_feats[num_cols]

y_lose = np.zeros(len(flipped_feats), dtype=int)

X_total = pd.concat([base_feats, flipped_feats], ignore_index=True)
y_total = np.concatenate([y_win, y_lose])

In [ ]:
#Daniel&Amaz
feat_transform = ColumnTransformer([
    ('std', StandardScaler(), num_cols),
    ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

pipe_rf = Pipeline([
    ('xform', feat_transform),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
])

cal_rf = CalibratedClassifierCV(estimator=pipe_rf, method='sigmoid', cv=5)

In [ ]:
#Daniel&Amaz
prob_preds = cross_val_predict(cal_rf, X_total, y_total, cv=5, method='predict_proba')[:, 1]
final_preds = (prob_preds >= 0.5).astype(int)

print("== Random Forest with Calibration ==")
print(classification_report(y_total, final_preds))
print("Brier Score:", brier_score_loss(y_total, prob_preds))


In [ ]:
#Daniel&Amaz
cal_y, cal_x = calibration_curve(y_total, prob_preds, n_bins=10)

plt.figure(figsize=(6, 6))
plt.plot(cal_x, cal_y, 'o-', label='RF Calibrated')
plt.plot([0, 1], [0, 1], '--', color='gray', label='Ideal')
plt.title("Calibrated Probability Curve")
plt.xlabel("Predicted Win Probability")
plt.ylabel("True Win Rate")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#Daniel
import joblib

cal_rf.fit(X_total, y_total)
joblib.dump(cal_rf, "model.pkl")
print("model.pkl saved")